In [39]:
import os, glob
import rasterio
import numpy as np

from morphology import *
from tqdm.notebook import tqdm

def get_list_fp_or_name(folder_dir, type_file = '*.tif', name=False):
        """
            Get all file path with file type is type_file.
        """
        list_fp = []
        list_name = []
        for file_ in glob.glob(os.path.join(folder_dir, type_file)):
            list_fp.append(file_)
            _, tail = os.path.split(file_)
            list_name.append(tail)
        if name:
            return list_name
        else:
            return list_fp


def export_green_cover_change(fp_mask_truoc, fp_mask_sau, value_green, out_fp_green_cover_change, export_tif=True):
    """
        Input:
                fp_mask_truoc = r""
                fp_mask_sau = r""
                value_green = 2
                out_fp_green_cover_change = r""
        Output: 
                La mask vs
                    1: la green to non-green
                    2: la non-green to green 
    """

    with rasterio.open(fp_mask_truoc) as src:
        mask_truoc = src.read()
        meta = src.meta
    with rasterio.open(fp_mask_sau) as src:
        mask_sau = src.read()

    # chuan hoa lai du lieu
    mask_truoc[mask_truoc != value_green] = 0
    mask_sau[mask_sau != value_green] = 0
    mask_truoc = mask_truoc.astype('int64')
    mask_sau = mask_sau.astype('int64')

    # hieu 2 mask
    mask_change = mask_truoc - mask_sau
    print(np.unique(mask_change),"^^")

    # green to non_green: la gia tri value
    mask_change[mask_change == value_green] = 1
    # non_green to green: la gia tri am
    mask_change[mask_change < 0] = 2

    mask_change = mask_change.astype('uint8')
    print(np.unique(mask_change),"^-^")
    if export_tif:
        with rasterio.open(out_fp_green_cover_change, 'w', **meta) as dst:
            dst.write(mask_change)
    
    return mask_change

In [40]:
def green_cover_change_all(input_truoc, input_sau, value_green, output_greencover_change, export_tif=True):
    """
        Chay green cover change
            INPUT: Co the la file hoac dir
            OUTPUT: Co the la file hoac dir
    """
    if os.path.isfile(input_truoc) and os.path.isfile(input_sau):
        if output_greencover_change[-4:] == '.tif':
            os.makedirs(os.path.dirname(output_greencover_change), exist_ok=True)
            mask_change = export_green_cover_change(input_truoc, input_sau, value_green, output_greencover_change, export_tif)
            return mask_change
        else:
            print('Output phai la file ".tif"')
    elif os.path.isdir(input_truoc) and os.path.isdir(input_sau):
        if output_greencover_change[-4:] == '.tif':
            print('Output phai la direction chu k phai la ".tif"')
        else:
            list_name = get_list_fp_or_name(input_truoc, name=True)
            os.makedirs(output_greencover_change, exist_ok=True)
            for fname in tqdm(list_name):
                fp_truoc = os.path.join(input_truoc, fname)
                fp_sau = os.path.join(input_sau, fname)
                fp_out_change = os.path.join(output_greencover_change, fname)
                export_green_cover_change(fp_truoc, fp_sau, value_green, fp_out_change, export_tif)
    else:
        print('dau vao co van de r')
    

In [60]:
input_truoc = r"/home/skm/SKM16/Tmp/green_cover_change/All_result/Classification/2019-AOI1_fn_final.tif"
input_sau = r"/home/skm/SKM16/Tmp/green_cover_change/All_result/Classification/2022-AOI1_fn_final.tif"
value_green = 2
remove_area = 1
output_greencover_change = f"/home/skm/SKM16/Tmp/green_cover_change/All_result/Greencover_Change/Change_RM_{remove_area}.tif"
mask_change = export_green_cover_change(input_truoc, input_sau, value_green, output_greencover_change, export_tif=False)
print(mask_change.shape)
print(np.unique(mask_change))
# remove area
mask_rm_small = mask_change.copy()[0]
mask_rm_small[mask_rm_small!=0]=255
mask_rm_small = remove_area_small(mask=mask_rm_small, area_maximum=remove_area, value_draw=0)
mask_rm_small[mask_rm_small!=0]=1
mask_rm_small = np.array([mask_rm_small])
print(np.unique(mask_change))
mask_change = mask_change*mask_rm_small
print(np.unique(mask_rm_small))
print(np.unique(mask_change))
with rasterio.open(input_truoc) as src:
        meta = src.meta
with rasterio.open(output_greencover_change, 'w', **meta) as dst:
    dst.write(mask_change)
    dst.write_colormap(1, {
                    0: (0,0,0, 0),
                    1: (185,180,8,0), #Green to Non
                    2: (63, 249, 128, 0), # Non to Green
                    }) 
print('Done')

[-2  0  2] ^^
[0 1 2] ^-^
(1, 3728, 3547)
[0 1 2]


Run contour:   0%|          | 0/40564 [00:00<?, ?it/s]

[0 1 2]
[0 1]
[0 1 2]
Done


## ADD color

In [7]:
# dst.write_colormap(1, {
#                 0: (0,0,0, 0),
#                 1: (34,139,34,0), #Green 
#                 2: (100, 149, 237, 0), #water
#                 3: (101,67,33, 0)}) #Buildup


## Matching rs tuy tung bai toan

In [35]:
import rasterio
import numpy as np
from morphology import *

"""nam 2019"""
# fp_mask_green = r"/home/skm/SKM16/Tmp/green_cover_change/2019-AOI1_fn_rs.tif"
# fp_mask_label_hieu = r"/home/skm/SKM16/Tmp/green_cover_change/2019-AOI1_fn_label.tif"
# fp_water = r"/home/skm/SKM16/Tmp/green_cover_change/2019-AOI1_fn_rs_water.tif"
# fp_can_giu = r"/home/skm/SKM16/Tmp/green_cover_change/mask_can_giu_2019.tif"
rs_2019 = r"/home/skm/SKM16/Tmp/green_cover_change/rs/2019-AOI1_fn_final.tif"
# with rasterio.open(fp_water) as src:
#     mask_water_model = src.read()
# with rasterio.open(fp_can_giu) as src:
#     mask_can_giu = src.read()  
#     mask_can_giu[fp_can_giu==255] = 1
# mask_rs = mask_water_model*mask_can_giu
# mask_rs[mask_rs!=0] = 3

# with rasterio.open(fp_mask_green) as src:
#     mask_green = src.read()
#     meta = src.meta
# with rasterio.open(fp_mask_label_hieu) as src:
#     mask_water = src.read()

# mask_water = mask_water[0]
# mask_water = opening(mask_water,1)
# mask_water = np.array([mask_water])

# mask_rs[mask_green!=0] = 2
# mask_rs[mask_water == 3] = 3
# mask_rs[mask_rs == 0] = 1
# print(np.unique(mask_rs))


"""nam 2022"""
# fp_mask_green = r"/home/skm/SKM16/Tmp/green_cover_change/2022-AOI1_fn_rs.tif"
# fp_mask_label_hieu = r"/home/skm/SKM16/Tmp/green_cover_change/2022-AOI1_fn_label.tif"
# fp_water = r"/home/skm/SKM16/Tmp/green_cover_change/2022-AOI1_fn_rs_water.tif"
# fp_can_giu = r"/home/skm/SKM16/Tmp/green_cover_change/mask_can_giu/2022-AOI1_fn_cangiu.tif"
# rs = r"/home/skm/SKM16/Tmp/green_cover_change/rs/2022-AOI1_fn_final.tif"
# fp_them_cua_2019 = r"/home/skm/SKM16/Tmp/green_cover_change/mask_can_giu/2019-AOI1_fn_final_Add_them.tif"

# with rasterio.open(rs_2019) as src:
#     water_2019 = src.read()
# with rasterio.open(fp_them_cua_2019) as src:
#     mask_giu_2019 = src.read()
#     mask_giu_2019[mask_giu_2019!=0] = 1

# with rasterio.open(fp_water) as src:
#     mask_water_model = src.read()
# with rasterio.open(fp_can_giu) as src:
#     mask_can_giu = src.read()  
#     mask_can_giu[fp_can_giu==255] = 1
# mask_rs = mask_water_model*mask_can_giu
# mask_rs[mask_rs!=0] = 3

# with rasterio.open(fp_mask_green) as src:
#     mask_green = src.read()
#     meta = src.meta
# with rasterio.open(fp_mask_label_hieu) as src:
#     mask_water = src.read()


# mask_water = mask_water[0]
# mask_water = opening(mask_water,1)
# mask_water = np.array([mask_water])

# mask_rs[mask_green!=0] = 2
# mask_rs[mask_water == 3] = 3
# mask_rs[mask_rs == 0] = 1
# print(np.unique(mask_rs))
# mask_add_2019 = water_2019*mask_giu_2019
# mask_rs[mask_add_2019==3] = 3

"""Chung"""
meta.update({'nodata': 1})
with  rasterio.open(rs,'w',**meta) as dst:
    dst.write(mask_rs)
    dst.write_colormap(1, {
                    1: (0,0,0, 0),
                    2: (34,139,34,0), #Green to Non
                    3: (100, 149, 237, 0), # Non to Green
                #     3: (101,67,33, 0) #Buildup
                    }) 

[1 2 3]


## Doi mau va gia tri anh

In [38]:
import rasterio
fp = r"/home/skm/SKM16/Tmp/green_cover_change/rs (copy)/2019-AOI1_fn_final.tif"
with rasterio.open(fp) as src:
    mask = src.read()
    meta = src.meta
mask[mask==0] = 1

meta.update({'nodata':0})
with  rasterio.open(fp,'w',**meta) as dst:
    dst.write(mask)
    dst.write_colormap(1, {
                    1: (228, 122, 16, 0), # BF
                    2: (34, 139, 34, 0), #Green
                    3: (17, 185, 219, 0), # Water
                #     3: (101,67,33, 0) #
                    }) 